In [ ]:
import os
import numpy as np
import shutil
import matplotlib
import matplotlib.pyplot as plt
from astropy import table
from astropy.io import ascii
import cmasher as cmr
import dynamite as dyn
from plotbin import display_pixels

In [ ]:
# # Get an estimate for the weights from losvd optimization (config files MUST have the same parspace)
# fname = 'F2_config11.yaml'
# c = dyn.config_reader.Configuration(fname,
#                                     reset_logging=True,
#                                     user_logfile='test_nnls',
#                                     reset_existing_output=True)
# _ = dyn.model_iterator.ModelIterator(config=c)
# best_model_idx = c.all_models.get_best_n_models_idx(n=1)[0]
# model = c.all_models.get_model_from_row(best_model_idx)
# parset = model.parset
# table.Table(model.parset).write('parset.ecsv', format='ascii.ecsv', overwrite=True)
# print(f'{parset=}')
# orblib = model.get_orblib()
# _ = model.get_weights(orblib)
# weights = model.weights
# print(f'{weights.shape=}')
# orblib.read_vel_histograms()
# print(f'{len(orblib.vel_histograms)=}\t{len(orblib.intrinsic_masses)=}\t{len(orblib.projected_masses)=}\t{orblib.n_orbs=}')
# shutil.copyfile(model.directory + 'orbit_weights.ecsv', 'weights.ecsv')

In [ ]:
# Read proper motion data
fname = 'F2_config11_with_pm.yaml'
c = dyn.config_reader.Configuration(fname,
                                    reset_logging=True,
                                    user_logfile='test_nnls',
                                    reset_existing_output=False)
stars = c.system.get_unique_triaxial_visible_component()
# print(f'stars.kinematic_data[0]:\n{stars.kinematic_data[0]}\n\nstars.kinematic_data[1]:\n{stars.kinematic_data[1]}')
_ = dyn.model_iterator.ModelIterator(config=c)

In [ ]:
p = dyn.plotter.Plotter(c)

In [ ]:
f = stars.kinematic_data[1].data
print(sorted(f.files))
print(f"{f['PM_2dhist'].shape=}, {f['PM_2dhist_sigma'].shape=}")
print(f"{f['PSF_sigma']=}, {f['PSF_sigma'].shape=}, {f['PSF_weight']=}, {f['PSF_weight'].shape=}")
print(f"{f['binID_dynamite'].shape=}")
print(f"{f['nstarbin'].shape=}")
print(f"{f['vxrange']=}, {f['vxrange'].shape=}, {f['vyrange']=}, {f['vyrange'].shape=}")
print(f"{f['xbin'].shape=}, {f['ybin'].shape=}")
# sanity check: dv <= sigma_global/4, boundaries at least 5*sigma_global
dict(f)

In [ ]:
vx_mean = stars.kinematic_data[1].as_histogram2d().get_mean()[0][0]  # first index: vx/vy, second index: orb_idx
vy_mean = stars.kinematic_data[1].as_histogram2d().get_mean()[1][0]  # first index: vx/vy, second index: orb_idx
x, y = stars.kinematic_data[1].data['xbin'], stars.kinematic_data[1].data['ybin']
u, v = vx_mean, vy_mean
plt.quiver(x, y, u, v, color='g')
plt.axis('equal')
print(f'{stars.kinematic_data[1].as_histogram2d().get_mean()[0].shape=}')

In [ ]:
h2d = stars.kinematic_data[1].as_histogram2d()
for sp_bin_idx in(11, 48, 75):
    print(f'{sp_bin_idx=}')
    _ = p.hist2d_plot(h2d, orb_idx=0, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=True)
h2d_global = dyn.kinematics.Histogram2D(xedg=h2d.xedg,
                                        y=np.sum(h2d.y, axis=3)[:,:,:,np.newaxis],
                                        normalise=False)
_ = p.hist2d_plot(h2d_global, orb_idx=0, sp_bin_idx=0, show_1d=True, empty_bins=True)

In [ ]:
sorted(stars.kinematic_data[1].data.files)

In [ ]:
os.chdir('/Users/maindl/Documents/DriveM/drtim/ClientsNonGAMS/DYNAMITE/dynamite/dev_tests/propermotions')
os.getcwd()

In [ ]:
parset = ascii.read('parset.ecsv')[0]
orb_weights = ascii.read('weights.ecsv')['weights']
print(f'{parset=}')
model = dyn.model.Model(config=c, parset=parset)
model.setup_directories()
orblib = model.get_orblib()
print(f'{orblib.parset=}')
orblib.read_vel_histograms()
print(f'{len(orblib.vel_histograms)=}\t{len(orblib.intrinsic_masses)=}\t{len(orblib.projected_masses)=}\t{orblib.n_orbs=}')

In [ ]:
for h in orblib.vel_histograms:
    print(f'{h}: {h.y.shape=}\n')
    mean = h.get_mean()
    sigma = h.get_sigma()
    if type(h.xedg)==tuple:
        print(f'{[x.shape for x in h.xedg]=}, {[x.shape for x in h.x]=}')
        print(f'{[x.shape for x in mean]=}\n{[x.shape for x in sigma]=}')
    else:
        print(f'{h.xedg.shape=}, {h.x.shape=}')
        print(f'{mean.shape=}\n{sigma.shape=}')

In [ ]:
# plot pm histograms for all spatial bins
hist2d = orblib.vel_histograms[1]
n_orbits = hist2d.y.shape[0]
n_bins = hist2d.y.shape[-1]

vx_range = [hist2d.x[0].min(), hist2d.x[0].max()]
vy_range = [hist2d.x[1].min(), hist2d.x[1].max()]
ratio = (vy_range[1]-vy_range[0]) / (vx_range[1]-vx_range[0])
print(f'{vx_range=}, {vy_range=}')

fig = plt.figure(figsize=(20, 20 * (n_bins // 4 // 4 + 2) * ratio))
for bin_idx in range(0, n_bins, 1):  ##################################
#    data = np.einsum('ijkl,i', hist2d.y, orb_weights)[:,:,bin_idx]
    data = np.dot(hist2d.y.T, orb_weights).T[:,:,bin_idx]
    ax = plt.subplot(n_bins // 4 + (1 if n_bins % 4 > 0 else 0), 4, bin_idx + 1)
    im = ax.imshow(data, aspect='equal', interpolation='bilinear', #cmap=cm.RdYlGn,
                   origin='lower', extent=vx_range + vy_range,
                   vmax=data.max(), vmin=0)
    fig.colorbar(im, ax=ax, shrink=1)
    ax.set_title(f'Bin {bin_idx}')

plt.show()

In [ ]:
# plot pm histograms for specific bin
hist2d = orblib.vel_histograms[1]
n_orbits = hist2d.y.shape[0]

bin_idx = 31

hist2d = orblib.vel_histograms[1]
vx_range = [hist2d.x[0].min(), hist2d.x[0].max()]
vy_range = [hist2d.x[1].min(), hist2d.x[1].max()]
ratio = (vy_range[1]-vy_range[0]) / (vx_range[1]-vx_range[0])
print(f'{vx_range=}, {vy_range=}')

fig = plt.figure(figsize=(20, 20 * (n_orbits // 4 // 4 + 2) * ratio))
for orb_idx in range(0, n_orbits, 1):  ##################################
    data = hist2d.y[orb_idx,:,:,bin_idx]
    ax = plt.subplot(n_orbits // 4 + (1 if n_orbits % 4 > 0 else 0), 4, orb_idx + 1)
    im = ax.imshow(data, aspect='equal', interpolation='bilinear', #cmap=cm.RdYlGn,
                   origin='lower', extent=vx_range + vy_range,
                   vmax=data.max(), vmin=0)
    fig.colorbar(im, ax=ax)
    ax.set_title(f'Bin {bin_idx}, orbit {orb_idx}')

plt.show()

In [ ]:
# plot the losvd and pm histograms' mean values
# mostly from plotter.py
orb_skip = 50
for i, orb_idx in enumerate(range(0, orblib.n_orbs, orb_skip)):

    def create_plot(dp_args, data):
        # get aperture and bin data
        x = dp_args['x']
        y = dp_args['y']
        dx = dp_args['dx']
        grid = dp_args['idx_bin_to_pix']
        angle_deg = dp_args['angle']
        # Only select the pixels that have a bin associated with them.
        s = np.ravel(np.where((grid >= 0)))
    
        #fhist, _ = np.histogram(grid[s], bins=len(data))
        #data = data / fhist
        # plot settings
        #data_min = min(data[grid[s]] / max(data))
        #data_max = max(data[grid[s]] / max(data))
        # The galaxy has NOT already rotated with PA to align major axis with x
        map1 = cmr.get_sub_cmap('twilight_shifted', 0.05, 0.6)
        kw_display_pixels1 = dict(pixelsize=dx,
                                  angle=angle_deg,
                                  colorbar=True,
                                  nticks=7,
                                  # cmap='sauron')
                                  cmap=map1)
        # PLOT THE DATA
        #plt.figure()
        #c_c = data[grid[s]] / max(data)
        c_c = data[grid[s]]
        display_pixels.display_pixels(x, y, c_c,
                                      vmin=min(data[grid[s]]), vmax=max(data[grid[s]]),
                                      label='velocity',
                                      **kw_display_pixels1)
        #plt.gca().set_title(title)
        #plt.show()

    fig = plt.figure(figsize=(20, 30))

    n_rows, n_cols = orblib.n_orbs // orb_skip + 1, 3

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 1)
    if len(orblib.vel_histograms) > 1:  # assuming that 1d histograms are first, if existing
        ax.set_title(f'Orbit {orb_idx}, losvd')
        mean = orblib.vel_histograms[0].get_mean()[orb_idx]
        create_plot(stars.kinematic_data[0].dp_args, mean)

    idx = 1 if len(orblib.vel_histograms) > 1 else 0  # assuming that 1d histograms are first, if existing

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 2)
    ax.set_title(f'Orbit {orb_idx}, pm x')
    mean = orblib.vel_histograms[idx].get_mean()[0][orb_idx]
    create_plot(stars.kinematic_data[idx].dp_args, mean)

    ax = plt.subplot(n_rows, n_cols, n_cols * i + 3)
    ax.set_title(f'Orbit {orb_idx}, pm y')
    mean = orblib.vel_histograms[idx].get_mean()[1][orb_idx]
    create_plot(stars.kinematic_data[idx].dp_args, mean)

In [ ]:
orb_idx = 336
sp_bin_idx = 71
_ = p.hist2d_plot(hist2d, orb_idx=orb_idx, sp_bin_idx=sp_bin_idx, show_1d=True, empty_bins=False)

In [ ]:
import datetime
datetime.datetime.now()